In [1]:
import mxnet as mx
from mxnet.gluon import nn
from mxnet.gluon.contrib import nn as nn_contrib
from mxnet import nd
from mxnet import gluon
import numpy as np
ctx = mx.gpu()

In [2]:
global alpha
alpha = 0.5
num_filters = int(32*alpha)

In [3]:
def dp_layer(nfilters, stride, expension_constant):
    out = nn.HybridSequential()
    out.add(nn.Conv2D(nfilters, 3, strides=stride, padding=1, groups=nfilters, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    out.add(nn.Conv2D(nfilters*expension_constant, 1, strides=1, padding=0, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    return out

In [4]:
from mxnet import gluon
def s16():
    out = nn.HybridSequential()
    # conv_0 layer
    out.add(nn.Conv2D(num_filters, 3, strides=2, padding=1, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    # conv_1 layer
    out.add(dp_layer(num_filters, 1, 2))
    # conv_2 layer
    out.add(dp_layer(num_filters*2, 2, 2))
    # conv_3 layer
    out.add(dp_layer(num_filters*4, 1, 1))
    out.add(nn.Conv2D(num_filters*4, 3, strides=2, padding=1, groups=num_filters*4, use_bias=False))
    out.hybridize()
    return out

def s32():
    out = nn.HybridSequential()
    # from last layer
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    out.add(nn.Conv2D(num_filters*8, 1, strides=1, padding=0, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    # conv_4_layer
    out.add(dp_layer(num_filters*8, 1, 1))
    out.add(nn.Conv2D(num_filters*8, 3, strides=2, padding=1, groups=num_filters*8, use_bias=False))
    out.hybridize()
    return out

def fc():
    out = nn.HybridSequential()
    # from last layer
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    out.add(nn.Conv2D(num_filters*16, 1, strides=1, padding=0, use_bias=False))
    out.add(nn.BatchNorm(use_global_stats=False, epsilon=1e-05, momentum=0.9, axis=1))
    out.add(nn.Activation('relu'))
    # conv_5_layer
    out.add(dp_layer(num_filters*16, 1, 1))
    # conv_6_layer
    out.add(dp_layer(num_filters*16, 1, 1))
    # conv_7_layer
    out.add(dp_layer(num_filters*16, 1, 1))
    # conv_8_layer
    out.add(dp_layer(num_filters*16, 1, 1))
    # conv_9_layer
    out.add(dp_layer(num_filters*16, 1, 1))
    # conv_10_layer
    out.add(dp_layer(num_filters*16, 2, 2))
    # conv_11_layer
    out.add(dp_layer(num_filters*32, 1, 1))
    out.add(nn.GlobalAvgPool2D())
    out.add(nn.Flatten())
    out.add(nn.Dense(1000))
    out.hybridize()
    return out

In [5]:
s16 = s16()
s16.initialize()
s32 = s32()
s32.initialize()
fc = fc()
fc.initialize()
x = nd.zeros((2, 3, 256, 256))
s16(x)
s32(x)
fc(x)
s16.save_parameters("process/s16.params")
s32.save_parameters("process/s32.params")
fc.save_parameters("process/fc.params")

In [6]:
s16_params = mx.nd.load("process/s16.params")
sorted(s16_params.keys())

['0.weight',
 '1.beta',
 '1.gamma',
 '1.running_mean',
 '1.running_var',
 '3.0.weight',
 '3.1.beta',
 '3.1.gamma',
 '3.1.running_mean',
 '3.1.running_var',
 '3.3.weight',
 '3.4.beta',
 '3.4.gamma',
 '3.4.running_mean',
 '3.4.running_var',
 '4.0.weight',
 '4.1.beta',
 '4.1.gamma',
 '4.1.running_mean',
 '4.1.running_var',
 '4.3.weight',
 '4.4.beta',
 '4.4.gamma',
 '4.4.running_mean',
 '4.4.running_var',
 '5.0.weight',
 '5.1.beta',
 '5.1.gamma',
 '5.1.running_mean',
 '5.1.running_var',
 '5.3.weight',
 '5.4.beta',
 '5.4.gamma',
 '5.4.running_mean',
 '5.4.running_var',
 '6.weight']

In [7]:
s32_params = mx.nd.load("process/s32.params")
sorted(s32_params.keys())

['0.beta',
 '0.gamma',
 '0.running_mean',
 '0.running_var',
 '2.weight',
 '3.beta',
 '3.gamma',
 '3.running_mean',
 '3.running_var',
 '5.0.weight',
 '5.1.beta',
 '5.1.gamma',
 '5.1.running_mean',
 '5.1.running_var',
 '5.3.weight',
 '5.4.beta',
 '5.4.gamma',
 '5.4.running_mean',
 '5.4.running_var',
 '6.weight']

In [8]:
fc_params = mx.nd.load("process/fc.params")
sorted(fc_params.keys())

['0.beta',
 '0.gamma',
 '0.running_mean',
 '0.running_var',
 '10.0.weight',
 '10.1.beta',
 '10.1.gamma',
 '10.1.running_mean',
 '10.1.running_var',
 '10.3.weight',
 '10.4.beta',
 '10.4.gamma',
 '10.4.running_mean',
 '10.4.running_var',
 '11.0.weight',
 '11.1.beta',
 '11.1.gamma',
 '11.1.running_mean',
 '11.1.running_var',
 '11.3.weight',
 '11.4.beta',
 '11.4.gamma',
 '11.4.running_mean',
 '11.4.running_var',
 '14.bias',
 '14.weight',
 '2.weight',
 '3.beta',
 '3.gamma',
 '3.running_mean',
 '3.running_var',
 '5.0.weight',
 '5.1.beta',
 '5.1.gamma',
 '5.1.running_mean',
 '5.1.running_var',
 '5.3.weight',
 '5.4.beta',
 '5.4.gamma',
 '5.4.running_mean',
 '5.4.running_var',
 '6.0.weight',
 '6.1.beta',
 '6.1.gamma',
 '6.1.running_mean',
 '6.1.running_var',
 '6.3.weight',
 '6.4.beta',
 '6.4.gamma',
 '6.4.running_mean',
 '6.4.running_var',
 '7.0.weight',
 '7.1.beta',
 '7.1.gamma',
 '7.1.running_mean',
 '7.1.running_var',
 '7.3.weight',
 '7.4.beta',
 '7.4.gamma',
 '7.4.running_mean',
 '7.4.runni

In [9]:
mn_org_params = mx.nd.load("process/mobilenet0.5-8e9d539c.params")
sorted(mn_org_params.keys())

['batchnorm0_beta',
 'batchnorm0_gamma',
 'batchnorm0_running_mean',
 'batchnorm0_running_var',
 'batchnorm10_beta',
 'batchnorm10_gamma',
 'batchnorm10_running_mean',
 'batchnorm10_running_var',
 'batchnorm11_beta',
 'batchnorm11_gamma',
 'batchnorm11_running_mean',
 'batchnorm11_running_var',
 'batchnorm12_beta',
 'batchnorm12_gamma',
 'batchnorm12_running_mean',
 'batchnorm12_running_var',
 'batchnorm13_beta',
 'batchnorm13_gamma',
 'batchnorm13_running_mean',
 'batchnorm13_running_var',
 'batchnorm14_beta',
 'batchnorm14_gamma',
 'batchnorm14_running_mean',
 'batchnorm14_running_var',
 'batchnorm15_beta',
 'batchnorm15_gamma',
 'batchnorm15_running_mean',
 'batchnorm15_running_var',
 'batchnorm16_beta',
 'batchnorm16_gamma',
 'batchnorm16_running_mean',
 'batchnorm16_running_var',
 'batchnorm17_beta',
 'batchnorm17_gamma',
 'batchnorm17_running_mean',
 'batchnorm17_running_var',
 'batchnorm18_beta',
 'batchnorm18_gamma',
 'batchnorm18_running_mean',
 'batchnorm18_running_var',
 'ba

In [10]:
s16_org_params = dict()
s16_org_params['0.weight']=mn_org_params['conv0_weight']
s16_org_params['1.beta']=mn_org_params['batchnorm0_beta']
s16_org_params['1.gamma']=mn_org_params['batchnorm0_gamma']
s16_org_params['1.running_mean']=mn_org_params['batchnorm0_running_mean']
s16_org_params['1.running_var']=mn_org_params['batchnorm0_running_var']
s16_org_params['3.0.weight']=mn_org_params['conv1_weight']
s16_org_params['3.1.beta']=mn_org_params['batchnorm1_beta']
s16_org_params['3.1.gamma']=mn_org_params['batchnorm1_gamma']
s16_org_params['3.1.running_mean']=mn_org_params['batchnorm1_running_mean']
s16_org_params['3.1.running_var']=mn_org_params['batchnorm1_running_var']
s16_org_params['3.3.weight']=mn_org_params['conv2_weight']
s16_org_params['3.4.beta']=mn_org_params['batchnorm2_beta']
s16_org_params['3.4.gamma']=mn_org_params['batchnorm2_gamma']
s16_org_params['3.4.running_mean']=mn_org_params['batchnorm2_running_mean']
s16_org_params['3.4.running_var']=mn_org_params['batchnorm2_running_var']
s16_org_params['4.0.weight']=mn_org_params['conv3_weight']
s16_org_params['4.1.beta']=mn_org_params['batchnorm3_beta']
s16_org_params['4.1.gamma']=mn_org_params['batchnorm3_gamma']
s16_org_params['4.1.running_mean']=mn_org_params['batchnorm3_running_mean']
s16_org_params['4.1.running_var']=mn_org_params['batchnorm3_running_var']
s16_org_params['4.3.weight']=mn_org_params['conv4_weight']
s16_org_params['4.4.beta']=mn_org_params['batchnorm4_beta']
s16_org_params['4.4.gamma']=mn_org_params['batchnorm4_gamma']
s16_org_params['4.4.running_mean']=mn_org_params['batchnorm4_running_mean']
s16_org_params['4.4.running_var']=mn_org_params['batchnorm4_running_var']
s16_org_params['5.0.weight']=mn_org_params['conv5_weight']
s16_org_params['5.1.beta']=mn_org_params['batchnorm5_beta']
s16_org_params['5.1.gamma']=mn_org_params['batchnorm5_gamma']
s16_org_params['5.1.running_mean']=mn_org_params['batchnorm5_running_mean']
s16_org_params['5.1.running_var']=mn_org_params['batchnorm5_running_var']
s16_org_params['5.3.weight']=mn_org_params['conv6_weight']
s16_org_params['5.4.beta']=mn_org_params['batchnorm6_beta']
s16_org_params['5.4.gamma']=mn_org_params['batchnorm6_gamma']
s16_org_params['5.4.running_mean']=mn_org_params['batchnorm6_running_mean']
s16_org_params['5.4.running_var']=mn_org_params['batchnorm6_running_var']
s16_org_params['6.weight']=mn_org_params['conv7_weight']

In [11]:
s32_org_params = dict()
s32_org_params['0.beta']=mn_org_params['batchnorm7_beta']
s32_org_params['0.gamma']=mn_org_params['batchnorm7_gamma']
s32_org_params['0.running_mean']=mn_org_params['batchnorm7_running_mean']
s32_org_params['0.running_var']=mn_org_params['batchnorm7_running_var']
s32_org_params['2.weight']=mn_org_params['conv8_weight']
s32_org_params['3.beta']=mn_org_params['batchnorm8_beta']
s32_org_params['3.gamma']=mn_org_params['batchnorm8_gamma']
s32_org_params['3.running_mean']=mn_org_params['batchnorm8_running_mean']
s32_org_params['3.running_var']=mn_org_params['batchnorm8_running_var']
s32_org_params['5.0.weight']=mn_org_params['conv9_weight']
s32_org_params['5.1.beta']=mn_org_params['batchnorm9_beta']
s32_org_params['5.1.gamma']=mn_org_params['batchnorm9_gamma']
s32_org_params['5.1.running_mean']=mn_org_params['batchnorm9_running_mean']
s32_org_params['5.1.running_var']=mn_org_params['batchnorm9_running_var']
s32_org_params['5.3.weight']=mn_org_params['conv10_weight']
s32_org_params['5.4.beta']=mn_org_params['batchnorm10_beta']
s32_org_params['5.4.gamma']=mn_org_params['batchnorm10_gamma']
s32_org_params['5.4.running_mean']=mn_org_params['batchnorm10_running_mean']
s32_org_params['5.4.running_var']=mn_org_params['batchnorm10_running_var']
s32_org_params['6.weight']=mn_org_params['conv11_weight']

In [12]:
fc_org_params = dict()
fc_org_params['0.beta']=mn_org_params['batchnorm11_beta']
fc_org_params['0.gamma']=mn_org_params['batchnorm11_gamma']
fc_org_params['0.running_mean']=mn_org_params['batchnorm11_running_mean']
fc_org_params['0.running_var']=mn_org_params['batchnorm11_running_var']
fc_org_params['2.weight']=mn_org_params['conv12_weight']
fc_org_params['3.beta']=mn_org_params['batchnorm12_beta']
fc_org_params['3.gamma']=mn_org_params['batchnorm12_gamma']
fc_org_params['3.running_mean']=mn_org_params['batchnorm12_running_mean']
fc_org_params['3.running_var']=mn_org_params['batchnorm12_running_var']
fc_org_params['5.0.weight']=mn_org_params['conv13_weight']
fc_org_params['5.1.beta']=mn_org_params['batchnorm13_beta']
fc_org_params['5.1.gamma']=mn_org_params['batchnorm13_gamma']
fc_org_params['5.1.running_mean']=mn_org_params['batchnorm13_running_mean']
fc_org_params['5.1.running_var']=mn_org_params['batchnorm13_running_var']
fc_org_params['5.3.weight']=mn_org_params['conv14_weight']
fc_org_params['5.4.beta']=mn_org_params['batchnorm14_beta']
fc_org_params['5.4.gamma']=mn_org_params['batchnorm14_gamma']
fc_org_params['5.4.running_mean']=mn_org_params['batchnorm14_running_mean']
fc_org_params['5.4.running_var']=mn_org_params['batchnorm14_running_var']
fc_org_params['6.0.weight']=mn_org_params['conv15_weight']
fc_org_params['6.1.beta']=mn_org_params['batchnorm15_beta']
fc_org_params['6.1.gamma']=mn_org_params['batchnorm15_gamma']
fc_org_params['6.1.running_mean']=mn_org_params['batchnorm15_running_mean']
fc_org_params['6.1.running_var']=mn_org_params['batchnorm15_running_var']
fc_org_params['6.3.weight']=mn_org_params['conv16_weight']
fc_org_params['6.4.beta']=mn_org_params['batchnorm16_beta']
fc_org_params['6.4.gamma']=mn_org_params['batchnorm16_gamma']
fc_org_params['6.4.running_mean']=mn_org_params['batchnorm16_running_mean']
fc_org_params['6.4.running_var']=mn_org_params['batchnorm16_running_var']
fc_org_params['7.0.weight']=mn_org_params['conv17_weight']
fc_org_params['7.1.beta']=mn_org_params['batchnorm17_beta']
fc_org_params['7.1.gamma']=mn_org_params['batchnorm17_gamma']
fc_org_params['7.1.running_mean']=mn_org_params['batchnorm17_running_mean']
fc_org_params['7.1.running_var']=mn_org_params['batchnorm17_running_var']
fc_org_params['7.3.weight']=mn_org_params['conv18_weight']
fc_org_params['7.4.beta']=mn_org_params['batchnorm18_beta']
fc_org_params['7.4.gamma']=mn_org_params['batchnorm18_gamma']
fc_org_params['7.4.running_mean']=mn_org_params['batchnorm18_running_mean']
fc_org_params['7.4.running_var']=mn_org_params['batchnorm18_running_var']
fc_org_params['8.0.weight']=mn_org_params['conv19_weight']
fc_org_params['8.1.beta']=mn_org_params['batchnorm19_beta']
fc_org_params['8.1.gamma']=mn_org_params['batchnorm19_gamma']
fc_org_params['8.1.running_mean']=mn_org_params['batchnorm19_running_mean']
fc_org_params['8.1.running_var']=mn_org_params['batchnorm19_running_var']
fc_org_params['8.3.weight']=mn_org_params['conv20_weight']
fc_org_params['8.4.beta']=mn_org_params['batchnorm20_beta']
fc_org_params['8.4.gamma']=mn_org_params['batchnorm20_gamma']
fc_org_params['8.4.running_mean']=mn_org_params['batchnorm20_running_mean']
fc_org_params['8.4.running_var']=mn_org_params['batchnorm20_running_var']
fc_org_params['9.0.weight']=mn_org_params['conv21_weight']
fc_org_params['9.1.beta']=mn_org_params['batchnorm21_beta']
fc_org_params['9.1.gamma']=mn_org_params['batchnorm21_gamma']
fc_org_params['9.1.running_mean']=mn_org_params['batchnorm21_running_mean']
fc_org_params['9.1.running_var']=mn_org_params['batchnorm21_running_var']
fc_org_params['9.3.weight']=mn_org_params['conv22_weight']
fc_org_params['9.4.beta']=mn_org_params['batchnorm22_beta']
fc_org_params['9.4.gamma']=mn_org_params['batchnorm22_gamma']
fc_org_params['9.4.running_mean']=mn_org_params['batchnorm22_running_mean']
fc_org_params['9.4.running_var']=mn_org_params['batchnorm22_running_var']
fc_org_params['10.0.weight']=mn_org_params['conv23_weight']
fc_org_params['10.1.beta']=mn_org_params['batchnorm23_beta']
fc_org_params['10.1.gamma']=mn_org_params['batchnorm23_gamma']
fc_org_params['10.1.running_mean']=mn_org_params['batchnorm23_running_mean']
fc_org_params['10.1.running_var']=mn_org_params['batchnorm23_running_var']
fc_org_params['10.3.weight']=mn_org_params['conv24_weight']
fc_org_params['10.4.beta']=mn_org_params['batchnorm24_beta']
fc_org_params['10.4.gamma']=mn_org_params['batchnorm24_gamma']
fc_org_params['10.4.running_mean']=mn_org_params['batchnorm24_running_mean']
fc_org_params['10.4.running_var']=mn_org_params['batchnorm24_running_var']
fc_org_params['11.0.weight']=mn_org_params['conv25_weight']
fc_org_params['11.1.beta']=mn_org_params['batchnorm25_beta']
fc_org_params['11.1.gamma']=mn_org_params['batchnorm25_gamma']
fc_org_params['11.1.running_mean']=mn_org_params['batchnorm25_running_mean']
fc_org_params['11.1.running_var']=mn_org_params['batchnorm25_running_var']
fc_org_params['11.3.weight']=mn_org_params['conv26_weight']
fc_org_params['11.4.beta']=mn_org_params['batchnorm26_beta']
fc_org_params['11.4.gamma']=mn_org_params['batchnorm26_gamma']
fc_org_params['11.4.running_mean']=mn_org_params['batchnorm26_running_mean']
fc_org_params['11.4.running_var']=mn_org_params['batchnorm26_running_var']
fc_org_params['14.bias']=mn_org_params['dense0_bias']
fc_org_params['14.weight']=mn_org_params['dense0_weight']

In [13]:
mx.nd.save("process/mobilenet_0_5_s16.params",s16_org_params)
mx.nd.save("process/mobilenet_0_5_s32.params",s32_org_params)
mx.nd.save("process/mobilenet_0_5_fc.params",fc_org_params)